In [2]:
import csv
import pandas as pd
from sklearn.linear_model import LogisticRegression 
from sklearn.preprocessing import LabelEncoder
LE=LabelEncoder()
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

In [3]:
segment3_workflow1=pd.read_csv('segment3_workflowS1.csv')
segment3_workflow1.CASE_ID.mode()

0    1175521|568653|866936
dtype: object

In [4]:
segment3_workflow1.head()

,CASE_ID,WRKFLW_TSK_MAP_GROUPED,WRKFLW_TSK_MAP_DETAILED,PATH_CRDATETIME,ITEM_CRUSER,WRKFLW_ORG_TYPE_CRUSER,STATUS_ORDER,LATEST_STATUS,WRKFLW_PROCESS,QUERY_PART,...,ORIG_SPRD_CD,ORIG_SPRD_NM,SPK_CAT_TYPE_CD,BROAD_COURSE_LEVEL,DEGREETYPE,LIABILITY_CATEGORY_CD_RECODE,ORIG_LIABILITY_CATEGORY_CD_RECODE,OFFER_RELEASED_FG,OFFER_OUTCOME_CD,OFFER_CONDITIONS
0,1067449|424909|657898,Submit,Submit,26/11/2018 17:58:06,GUEST,Student,15,N,1. App to Assessment,Part 1 - Application ID Workflow,...,STYI,Start Year Intake,MCW,HDCW,Full Degree,AF,AF,Y,$ACC,Y
1,1067449|424909|657898,AR - Satisfied - previous or current study,AR - Satisfied - Academic results for prior te...,7/12/2018 10:51:18,ATSCHORN,Academic Services,14,N,1. App to Assessment,Part 6 - Requirement Verify,...,STYI,Start Year Intake,MCW,HDCW,Full Degree,AF,AF,Y,$ACC,Y
2,1067449|424909|657898,AR - Not Satisfied - ELR,AR - Not Satisfied - English language proficiency,7/12/2018 10:51:38,ATSCHORN,Academic Services,13,N,1. App to Assessment,Part 6 - Requirement Verify,...,STYI,Start Year Intake,MCW,HDCW,Full Degree,AF,AF,Y,$ACC,Y
3,1067449|424909|657898,OC - Created - Other,OC - Created - English language requirements,7/12/2018 10:51:46,ATSCHORN,Academic Services,12,N,1. App to Assessment,Part 5 - Requirement Create,...,STYI,Start Year Intake,MCW,HDCW,Full Degree,AF,AF,Y,$ACC,Y
4,1067449|424909|657898,AR - Created - Other,AR - Created - Assessment information (Coursew...,7/12/2018 11:00:43,ATSCHORN,Academic Services,11,N,1. App to Assessment,Part 5 - Requirement Create,...,STYI,Start Year Intake,MCW,HDCW,Full Degree,AF,AF,Y,$ACC,Y


In [5]:
segment3_workflow1['WRKFLW_TSK_MAP_DETAILED'].nunique()

112

In [6]:
segment3_workflow1=segment3_workflow1.drop(['WRKFLW_TSK_MAP_GROUPED', 'PATH_CRDATETIME','ITEM_CRUSER','WRKFLW_ORG_TYPE_CRUSER',
                         'STATUS_ORDER','LATEST_STATUS','WRKFLW_PROCESS','QUERY_PART','REFERRAL_ID',
                         'APPLICATION_ID','STU_ID','STU_CITIZEN_CD','STU_CITIZEN_NM','STU_BIRTH_DT',
                         'STU_ABOR_TSI_CD','STU_ABOR_TSI_NM','STU_BIRTH_CNTRY_CD','STU_BIRTH_CNTRY_NM',
                         'STU_CTZN_CNTRY_CD','STU_CTZN_CNTRY_NM','SUBMISSION_METHOD','ENTRY_TYPE',
                         'ASSESSMENT_DECISION_SCORE','ASSESSMENT_DECISION_NM','ASSESSMENT_OUTCOME_NM',
                         'LAST_COUNTRY_EDUCATION','PREV_INST_NM_MAIN','PREV_INST_NM_ALT1','PREV_INST_NM_ALT2',
                        'PREFERENCE_NO','SPK_CD','SPK_FULL_TITLE','ORG_UNIT_CD','AVAIL_ORG_UNIT_NM','SUPPORTING_ORG',
                        'STUDY_MODE_CD','RETURNING_DEFERRAL','CURR_AVAIL_YR','CURR_STUDY_PERIOD','CURR_SPRD_CD',
                        'CURR_SPRD_NM','ORIG_AVAIL_YR','ORIG_STUDY_PERIOD','ORIG_SPRD_CD','ORIG_SPRD_NM',
                         'SPK_CAT_TYPE_CD','BROAD_COURSE_LEVEL','DEGREETYPE','LIABILITY_CATEGORY_CD_RECODE',
                        'ORIG_LIABILITY_CATEGORY_CD_RECODE','OFFER_RELEASED_FG','OFFER_OUTCOME_CD','OFFER_CONDITIONS',
                        'APPLICATION_LINE_ID'], axis=1)
segment3_workflow1.head()

,CASE_ID,WRKFLW_TSK_MAP_DETAILED
0,1067449|424909|657898,Submit
1,1067449|424909|657898,AR - Satisfied - Academic results for prior te...
2,1067449|424909|657898,AR - Not Satisfied - English language proficiency
3,1067449|424909|657898,OC - Created - English language requirements
4,1067449|424909|657898,AR - Created - Assessment information (Coursew...


In [7]:
numOfWorkflowProcesses=segment3_workflow1.groupby('CASE_ID').CASE_ID.count()
numOfWorkflowProcesses.head()

CASE_ID
1000727|431447|668147     3
1001829|336678|526826     3
1001829|336678|526827     3
1001877|336757|526956     4
1002271|337186|527523    10
Name: CASE_ID, dtype: int64

In [8]:
counts = segment3_workflow1['CASE_ID'].value_counts()

In [9]:
counts.head()

1175521|568653|866936    19
1075305|437160|675799    18
1125321|497557|764055    17
1086364|448364|689547    17
1045292|395605|615665    17
Name: CASE_ID, dtype: int64

In [12]:
filtered = counts[counts == 5]
seg3_count1=segment3_workflow1[segment3_workflow1['CASE_ID'].isin(filtered.index)]

In [13]:
seg3_count1['CASE_ID'].nunique()

86

In [14]:
seg3_count1.head()

,CASE_ID,WRKFLW_TSK_MAP_DETAILED
15,1021374|367597|573577,Submit
16,1021374|367597|573577,AR - Satisfied - Academic results for prior te...
17,1021374|367597|573577,AR - Satisfied - Transcript û My eQuals
18,1021374|367597|573577,AR - Satisfied - Credit for previous study
19,1021374|367597|573577,Send for Assessment


In [15]:
y = pd.get_dummies(seg3_count1.WRKFLW_TSK_MAP_DETAILED)
y=y[:0]

In [16]:
y

,AR - Created - Assessment information (Coursework) (1),AR - Created - Further information (1),AR - Created - Further information - Official Transcripts,AR - Created - University of Melbourne Statement of Results / transcript,AR - Not Satisfied - English language proficiency,AR - Requested - Academic results for prior tertiary studies,AR - Requested - Further information (1),AR - Reviewed - Academic results for prior tertiary studies,AR - Reviewed - Curriculum Vitae and professional experience,AR - Satisfied - Academic results for prior tertiary studies,...,AR - Satisfied - Further information - Official Transcripts,AR - Satisfied - Graduate Access Melbourne,AR - Satisfied - Transcript û My eQuals,AR - Satisfied - University of Melbourne Statement of Results / transcript,Assessment Decision - Qualified,OC - Created - English language requirements,Put Triage On-Hold,Release Triage Hold,Send for Assessment,Submit


In [17]:
y["CASE_ID"]=""

In [18]:
col_name="CASE_ID"
first_col = y.pop(col_name)

In [19]:
y.insert(0, col_name, first_col)

In [20]:
y['CASE_ID']=seg3_count1['CASE_ID'].unique()

In [21]:
import numpy as np
y = y.replace(np.nan,0, regex=True)
y.head()

,CASE_ID,AR - Created - Assessment information (Coursework) (1),AR - Created - Further information (1),AR - Created - Further information - Official Transcripts,AR - Created - University of Melbourne Statement of Results / transcript,AR - Not Satisfied - English language proficiency,AR - Requested - Academic results for prior tertiary studies,AR - Requested - Further information (1),AR - Reviewed - Academic results for prior tertiary studies,AR - Reviewed - Curriculum Vitae and professional experience,...,AR - Satisfied - Further information - Official Transcripts,AR - Satisfied - Graduate Access Melbourne,AR - Satisfied - Transcript û My eQuals,AR - Satisfied - University of Melbourne Statement of Results / transcript,Assessment Decision - Qualified,OC - Created - English language requirements,Put Triage On-Hold,Release Triage Hold,Send for Assessment,Submit
0,1021374|367597|573577,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1065704|422287|653985,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,760284|315130|499078,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1068696|426431|660784,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1063862|419463|673689,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
dict_1=seg3_count1.to_dict(orient='list')

In [23]:
case_id = dict_1['CASE_ID']

In [24]:
workflow = dict_1['WRKFLW_TSK_MAP_DETAILED']

In [25]:
print(type(workflow))
print(len(workflow))

<class 'list'>
430


In [26]:
from collections import defaultdict

case_wf_dict = defaultdict(list)

for i in range(len(case_id)):
    item = case_id[i]
    case_wf_dict[item].append(workflow[i])
print(len(case_wf_dict))

86


In [27]:
y = y.set_index('CASE_ID')

In [28]:
y.head()

,AR - Created - Assessment information (Coursework) (1),AR - Created - Further information (1),AR - Created - Further information - Official Transcripts,AR - Created - University of Melbourne Statement of Results / transcript,AR - Not Satisfied - English language proficiency,AR - Requested - Academic results for prior tertiary studies,AR - Requested - Further information (1),AR - Reviewed - Academic results for prior tertiary studies,AR - Reviewed - Curriculum Vitae and professional experience,AR - Satisfied - Academic results for prior tertiary studies,...,AR - Satisfied - Further information - Official Transcripts,AR - Satisfied - Graduate Access Melbourne,AR - Satisfied - Transcript û My eQuals,AR - Satisfied - University of Melbourne Statement of Results / transcript,Assessment Decision - Qualified,OC - Created - English language requirements,Put Triage On-Hold,Release Triage Hold,Send for Assessment,Submit
CASE_ID,,,,,,,,,,,,,,,,,,,,,
1021374|367597|573577,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1065704|422287|653985,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
760284|315130|499078,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1068696|426431|660784,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1063862|419463|673689,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
y_col_name = y.columns.values.tolist()
print(y_col_name)

['AR - Created - Assessment information (Coursework) (1)', 'AR - Created - Further information (1)', 'AR - Created - Further information - Official Transcripts', 'AR - Created - University of Melbourne Statement of Results / transcript', 'AR - Not Satisfied - English language proficiency', 'AR - Requested - Academic results for prior tertiary studies', 'AR - Requested - Further information (1)', 'AR - Reviewed - Academic results for prior tertiary studies', 'AR - Reviewed - Curriculum Vitae and professional experience', 'AR - Satisfied - Academic results for prior tertiary studies', 'AR - Satisfied - Advanced Standing', 'AR - Satisfied - Credit for previous study', 'AR - Satisfied - Current enrolment at the University of Melbourne', 'AR - Satisfied - Curriculum Vitae', 'AR - Satisfied - Curriculum Vitae and professional experience', 'AR - Satisfied - English language proficiency', 'AR - Satisfied - Entry points (Master of Information Systems)', 'AR - Satisfied - Entry points (Master of

In [30]:
for key in case_wf_dict.keys():
    for process in case_wf_dict[key]:
        if process in y_col_name:
            y.loc[key, process] = 1
        else:
            continue

In [31]:
pd.set_option('display.max_rows', None)

In [32]:
y['Submit'].head()

CASE_ID
1021374|367597|573577    1.0
1065704|422287|653985    1.0
760284|315130|499078     1.0
1068696|426431|660784    1.0
1063862|419463|673689    1.0
Name: Submit, dtype: float64

In [33]:
def movecol(df, cols_to_move=[], ref_col='', place='After'):
    
    cols = df.columns.tolist()
    if place == 'After':
        seg1 = cols[:list(cols).index(ref_col) + 1]
        seg2 = cols_to_move
    if place == 'Before':
        seg1 = cols[:list(cols).index(ref_col)]
        seg2 = cols_to_move + [ref_col]
    
    seg1 = [i for i in seg1 if i not in seg2]
    seg3 = [i for i in cols if i not in seg1 + seg2]
    
    return(df[seg1 + seg2 + seg3])

In [43]:
y.head()

,Submit,AR - Created - Assessment information (Coursework) (1),AR - Created - Further information (1),AR - Created - Further information - Official Transcripts,AR - Created - University of Melbourne Statement of Results / transcript,AR - Not Satisfied - English language proficiency,AR - Requested - Academic results for prior tertiary studies,AR - Requested - Further information (1),AR - Reviewed - Academic results for prior tertiary studies,AR - Reviewed - Curriculum Vitae and professional experience,AR - Satisfied - Academic results for prior tertiary studies,AR - Satisfied - Advanced Standing,AR - Satisfied - Credit for previous study,AR - Satisfied - Current enrolment at the University of Melbourne,AR - Satisfied - Curriculum Vitae,AR - Satisfied - Curriculum Vitae and professional experience,AR - Satisfied - English language proficiency,AR - Satisfied - Entry points (Master of Information Systems),AR - Satisfied - Entry points (Master of Information Technology),AR - Satisfied - Further information - Official Transcripts,AR - Satisfied - Graduate Access Melbourne,AR - Satisfied - Transcript û My eQuals,AR - Satisfied - University of Melbourne Statement of Results / transcript,Assessment Decision - Qualified,OC - Created - English language requirements,Put Triage On-Hold,Release Triage Hold,Send for Assessment
CASE_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1021374|367597|573577,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
1065704|422287|653985,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
760284|315130|499078,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1068696|426431|660784,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1063862|419463|673689,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [35]:
y = movecol(y, 
             cols_to_move=['Send for Assessment'], 
             ref_col='Submit',
             place='After')

In [36]:
y = movecol(y, 
             cols_to_move=['Submit'], 
             ref_col='AR - Created - Assessment information (Coursework) (1)',
             place='Before')

In [37]:
pd.set_option('display.max_columns',None)

In [38]:
y.shape

(86, 28)

In [39]:
y=y.fillna(0.0)

In [40]:
X=y.iloc[:,0:27].values
Y=y.iloc[:,27].values 

In [41]:
#spliting dataset
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size = 0.3,random_state=0)

In [46]:
# Linear SVM
svm = LinearSVC()
svm.fit(x_train, y_train)

LinearSVC()

In [47]:
svm_predict = svm.predict(x_test)
print("accuracy: {}%".format(round(accuracy_score(y_test, svm_predict)*100,2)))

accuracy: 100.0%


In [48]:
# Logistic Regression
lr = LogisticRegression()
lr.fit(x_train,y_train)

LogisticRegression()

In [49]:
lr_predict=lr.predict(x_test)
print("accuracy: {}%".format(round(accuracy_score(y_test, lr_predict)*100,2)))

accuracy: 100.0%


In [52]:
# K Nearest Neighbors
Knn = KNeighborsClassifier(n_neighbors=5)
Knn.fit(x_train,y_train)

KNeighborsClassifier()

In [53]:
knn_predict=Knn.predict(x_test)
print("accuracy: {}%".format(round(accuracy_score(y_test, knn_predict)*100,2)))

accuracy: 96.15%
